In [1]:
# A game company wants to create new customer definitions level based by using some features of its customers, 
# create segments according to these new customer definitions, and estimate how much the new customers can earn on average 
# according to these segments.
# VARIABLES
# PRICE – Customer's spending amount
# SOURCE – The type of device the customer is connecting to
# SEX – Gender of the client
# COUNTRY – Country of the customer
# AGE – Age of the customer
import pandas as pd
pd.set_option("display.max_rows", None)
df = pd.read_csv("persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [2]:
df.shape

(5000, 5)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [4]:
# How many unique SOURCE are there? 
df["SOURCE"].nunique()

2

In [5]:
# What are their frequencies?
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [6]:
# How many unique PRICEs are there?
df["PRICE"].nunique()

6

In [7]:
# How many sales were realized from which PRICE?
df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [8]:
# How many sales from which country?
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="count")

,PRICE
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


In [9]:
# How much was earned in total from sales by country?
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="sum")

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [10]:
# What are the sales numbers by SOURCE types?
df.pivot_table(values="PRICE",index="SOURCE",aggfunc="count")

,PRICE
SOURCE,
android,2974
ios,2026


In [11]:
# What are the PRICE averages by country?
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="mean")

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [12]:
# What are the PRICE averages according to SOURCEs?
df.pivot_table(values="PRICE",index="SOURCE",aggfunc="mean")

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [13]:
# What are the PRICE averages in the COUNTRY-SOURCE breakdown?
df.pivot_table(values="PRICE",index=["COUNTRY", "SOURCE"],aggfunc="mean")

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [14]:
# What are the average earnings in breakdown of COUNTRY, SOURCE, SEX, AGE?
df.pivot_table(values="PRICE",index=["COUNTRY", "SOURCE", "SEX", "AGE"],aggfunc="mean").head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

In [15]:
# To see the output in the previous question better, apply the sort_values method in descending order of PRICE.
# Save the output as agg_df.
agg_df = df.pivot_table(values="PRICE",index=["COUNTRY", "SOURCE", "SEX", "AGE"],aggfunc="mean").sort_values("PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [16]:
# All variables except PRICE in the output of the third question are index names. Convert these names to variable names.
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [17]:
# Convert the numeric variable age to a categorical variable.
# Construct the intervals convincingly.
# '0-18', '19-23', '24-30', '31-40', '41-60', '60+' 
intervals = [0, 18, 23, 30, 40, 60, agg_df["AGE"].max()]
labels = ["0-18", "19-23", "24-30", "30-40", "40-60", "60+"]
agg_df["CAT_AGE"] = pd.cut(agg_df["AGE"], intervals, labels=labels)
agg_df.head(10)

,COUNTRY,SOURCE,SEX,AGE,PRICE,CAT_AGE
0,bra,android,male,46,59.0,40-60
1,usa,android,male,36,59.0,30-40
2,fra,android,female,24,59.0,24-30
3,usa,ios,male,32,54.0,30-40
4,deu,android,female,36,49.0,30-40
5,deu,android,female,51,49.0,40-60
6,deu,ios,male,20,49.0,19-23
7,usa,android,female,47,49.0,40-60
8,fra,ios,male,20,49.0,19-23
9,usa,ios,male,59,46.5,40-60


In [18]:
# You need to create the LEVEL_BASED variable by combining the observations from the output.
# CREATE customers_level_based (COUNTRY_SOURCE_SEX_CAT_AGE)
agg_df["LEVEL_BASED"] = agg_df[["COUNTRY", "SOURCE", "SEX", "CAT_AGE"]].agg(lambda x: '_'.join(x).upper(), axis=1)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,CAT_AGE,LEVEL_BASED
0,bra,android,male,46,59.0,40-60,BRA_ANDROID_MALE_40-60
1,usa,android,male,36,59.0,30-40,USA_ANDROID_MALE_30-40
2,fra,android,female,24,59.0,24-30,FRA_ANDROID_FEMALE_24-30
3,usa,ios,male,32,54.0,30-40,USA_IOS_MALE_30-40
4,deu,android,female,36,49.0,30-40,DEU_ANDROID_FEMALE_30-40


In [19]:
# DROP COUNTRY, SOURCE, SEX, AGE, CAT_AGE
agg_df = agg_df[["LEVEL_BASED", "PRICE"]]
agg_df.head(10)

,LEVEL_BASED,PRICE
0,BRA_ANDROID_MALE_40-60,59.0
1,USA_ANDROID_MALE_30-40,59.0
2,FRA_ANDROID_FEMALE_24-30,59.0
3,USA_IOS_MALE_30-40,54.0
4,DEU_ANDROID_FEMALE_30-40,49.0
5,DEU_ANDROID_FEMALE_40-60,49.0
6,DEU_IOS_MALE_19-23,49.0
7,USA_ANDROID_FEMALE_40-60,49.0
8,FRA_IOS_MALE_19-23,49.0
9,USA_IOS_MALE_40-60,46.5


In [20]:
# Calculate average price based on LEVEL_BASED to determine range
agg_df = agg_df.groupby("LEVEL_BASED").agg({"PRICE": "mean"})
agg_df = agg_df.reset_index()
agg_df.head(10)

,LEVEL_BASED,PRICE
0,BRA_ANDROID_FEMALE_0-18,35.645303
1,BRA_ANDROID_FEMALE_19-23,34.077340
2,BRA_ANDROID_FEMALE_24-30,33.863946
3,BRA_ANDROID_FEMALE_30-40,34.898326
4,BRA_ANDROID_FEMALE_40-60,36.737179
5,BRA_ANDROID_MALE_0-18,34.805861
6,BRA_ANDROID_MALE_19-23,31.673243
7,BRA_ANDROID_MALE_24-30,33.413919
8,BRA_ANDROID_MALE_30-40,34.327381
9,BRA_ANDROID_MALE_40-60,40.041667


In [21]:
agg_df["PRICE"].nunique()

100

In [22]:
# Check that each level_based user is unique.
agg_df["LEVEL_BASED"].value_counts()

BRA_ANDROID_FEMALE_0-18     1
FRA_ANDROID_MALE_0-18       1
TUR_IOS_FEMALE_30-40        1
TUR_IOS_FEMALE_24-30        1
TUR_IOS_FEMALE_19-23        1
TUR_IOS_FEMALE_0-18         1
TUR_ANDROID_MALE_40-60      1
TUR_ANDROID_MALE_30-40      1
TUR_ANDROID_MALE_24-30      1
TUR_ANDROID_MALE_19-23      1
TUR_ANDROID_MALE_0-18       1
TUR_ANDROID_FEMALE_40-60    1
TUR_ANDROID_FEMALE_30-40    1
TUR_ANDROID_FEMALE_24-30    1
TUR_ANDROID_FEMALE_19-23    1
TUR_ANDROID_FEMALE_0-18     1
FRA_IOS_MALE_40-60          1
FRA_IOS_MALE_30-40          1
FRA_IOS_MALE_24-30          1
FRA_IOS_MALE_19-23          1
FRA_IOS_MALE_0-18           1
FRA_IOS_FEMALE_30-40        1
FRA_IOS_FEMALE_24-30        1
FRA_IOS_FEMALE_0-18         1
FRA_ANDROID_MALE_40-60      1
FRA_ANDROID_MALE_30-40      1
FRA_ANDROID_MALE_24-30      1
TUR_IOS_FEMALE_40-60        1
TUR_IOS_MALE_0-18           1
TUR_IOS_MALE_19-23          1
USA_ANDROID_MALE_40-60      1
USA_IOS_MALE_30-40          1
USA_IOS_MALE_24-30          1
USA_IOS_MA

In [23]:
# Create 4 separate ranges to parse the averaged price variable according to customers level_based.
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4)
agg_df.head(10)

,LEVEL_BASED,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0-18,35.645303,"(34.016, 36.0]"
1,BRA_ANDROID_FEMALE_19-23,34.077340,"(34.016, 36.0]"
2,BRA_ANDROID_FEMALE_24-30,33.863946,"(32.015, 34.016]"
3,BRA_ANDROID_FEMALE_30-40,34.898326,"(34.016, 36.0]"
4,BRA_ANDROID_FEMALE_40-60,36.737179,"(36.0, 45.667]"
5,BRA_ANDROID_MALE_0-18,34.805861,"(34.016, 36.0]"
6,BRA_ANDROID_MALE_19-23,31.673243,"(18.999, 32.015]"
7,BRA_ANDROID_MALE_24-30,33.413919,"(32.015, 34.016]"
8,BRA_ANDROID_MALE_30-40,34.327381,"(34.016, 36.0]"
9,BRA_ANDROID_MALE_40-60,40.041667,"(36.0, 45.667]"


In [24]:
# Checking if there are 4 intervals
agg_df["SEGMENT"].nunique()

4

In [25]:
# Show the intervals
agg_df["SEGMENT"].unique().sort_values()

[(18.999, 32.015], (32.015, 34.016], (34.016, 36.0], (36.0, 45.667]]
Categories (4, interval[float64, right]): [(18.999, 32.015] < (32.015, 34.016] < (34.016, 36.0] < (36.0, 45.667]]

In [26]:
# Write a function that will match the range values of the potential Customer's age.
def age_interval(age):
    if age <= 18:
        return "0-18"
    elif age <= 23:
        return "19-23"
    elif age <= 30:
        return "24-30"
    elif age <= 40:
        return "30-40"
    elif age <= 60:
        return "40-60"
    else:
        return "60+"

In [27]:
# Identify a new potential customer
country = "TUR"
source = "ANDROID"
sex = "MALE"
age = 40
user = country+"_"+source+"_"+sex+"_"+age_interval(age)

In [28]:
# Show expected earnings information from potential customer.
average = agg_df[agg_df["LEVEL_BASED"] == user]["PRICE"]
segment = agg_df[agg_df["LEVEL_BASED"] == user]["SEGMENT"]
print("Average earnings expected from this user: " + average.to_string(index=False))
print("The expected revenue range from this user: " + segment.to_string(index=False))

Average earnings expected from this user: 29.0
The expected revenue range from this user: (18.999, 32.015]
Categories (4, interval[float64, right]): [(18.999, 32.015] < (32.015, 34.016] < (34.016, 36.0] < (36.0, 45.667]]


In [29]:
# Show expected earnings information from potential customer.
agg_df[agg_df["LEVEL_BASED"] == user]

,LEVEL_BASED,PRICE,SEGMENT
78,TUR_ANDROID_MALE_30-40,29.0,"(18.999, 32.015]"
